# Predictions with nasnet models

In [3]:
CD = "cd ../ &&"

In [4]:
from pathlib import Path
import sys
sys.path.insert(0, Path(".").absolute().parent.as_posix())

Validation

In [5]:
files = [
    "configs/val_probas/cv/nasnetlarge_350_resized_crop/fold_0.py",
    "configs/val_probas/cv/nasnetlarge_350_resized_crop/fold_1.py",
    "configs/val_probas/cv/nasnetlarge_350_resized_crop/fold_2.py",
    "configs/val_probas/cv/nasnetlarge_350_resized_crop/fold_3.py",
]

In [8]:
!{CD} python3 multi_predict.py {" ".join(files)}

Run multiple predictions


----- run configs/val_probas/cv/nasnetlarge_350_resized_crop/fold_0.py -----

--- iMaterialist 2018 : Inference --- 
Load config file ... 
Activated debug mode
2018-05-30 07:47:54,531|iMaterialist 2018: Inference|DEBUG| Setup tensorboard writer
2018-05-30 07:47:54,533|iMaterialist 2018: Inference|INFO| 
        Configuration file: configs/val_probas/cv/nasnetlarge_350_resized_crop/fold_0.py

        LOG_DIR: output/val_probas/cv/nasnetlarge_350_resized_crop/fold_0/20180530_0747
            
    	# Basic training configuration file
	from pathlib import Path
	from torch.utils.data import ConcatDataset
	from torchvision.transforms import RandomVerticalFlip, RandomHorizontalFlip
	from torchvision.transforms import RandomResizedCrop
	from torchvision.transforms import ToTensor, Normalize
	
	from common.dataset import FilesFromCsvDataset
	from common.data_loaders import get_data_loader, get_trainval_indices
	
	
	SEED = 1245
	DEBUG = True
	DEVICE = "cuda"
	
	OUTPUT_

In [17]:
from pathlib import Path
import numpy as np
import pandas as pd


OUTPUT_PATH = Path(".").absolute().parent / "output" / "val_probas" / "cv" / "nasnetlarge_350_resized_crop"

In [24]:
!ls {(OUTPUT_PATH / "fold_3").as_posix()}

20180529_2317


In [25]:
val_prediction_files = [
    OUTPUT_PATH / "fold_0" / "20180529_2153" / "probas.csv",
    OUTPUT_PATH / "fold_1" / "20180529_2221" / "probas.csv",
    OUTPUT_PATH / "fold_2" / "20180529_2249" / "probas.csv",
    OUTPUT_PATH / "fold_3" / "20180529_2317" / "probas.csv",
]

val_dfs = [pd.read_csv(f, index_col='id') for f in val_prediction_files]
df_probas = pd.concat(val_dfs, axis=0)

In [31]:
from PIL import Image
from common.dataset import FilesFromCsvDataset, TransformedDataset
from common.meta import get_imsize_and_targets


dataset = FilesFromCsvDataset("../output/filtered_val_dataset.csv")
dataset = TransformedDataset(dataset,
                             transforms=lambda x: (x, Image.open(x).size),
                             target_transforms=lambda l: l - 1)

df_imsize_targets = get_imsize_and_targets(dataset)
y_true = df_imsize_targets['target']

In [32]:
y_probas = df_probas.groupby('id').mean()
y_probas = y_probas.loc[y_true.index, :]

In [33]:
from sklearn.metrics import accuracy_score


y_pred = np.argmax(y_probas.values, axis=1)
val_error = 1.0 - accuracy_score(y_true, y_pred)
val_error

0.10968049594659035

Predict on test dataset

In [9]:
files = [
    "configs/test_probas/cv/nasnetlarge_350_resized_crop/fold_0.py",
    "configs/test_probas/cv/nasnetlarge_350_resized_crop/fold_1.py",
    "configs/test_probas/cv/nasnetlarge_350_resized_crop/fold_2.py",
    "configs/test_probas/cv/nasnetlarge_350_resized_crop/fold_3.py",
]

In [11]:
!{CD} python3 multi_predict.py {" ".join(files)}

In [13]:
from pathlib import Path
import numpy as np
import pandas as pd

In [14]:
OUTPUT_PATH = Path(".").absolute().parent / "output" / "test_probas" / "cv" / "nasnetlarge_350_resized_crop"

test_prediction_files = [
#     OUTPUT_PATH / "fold_0" / "20180529_2353" / "probas.csv",
#     OUTPUT_PATH / "fold_1" / "20180530_0048" / "probas.csv",
    OUTPUT_PATH / "fold_2" / "20180530_0143" / "probas.csv",
    OUTPUT_PATH / "fold_3" / "20180530_0239" / "probas.csv",
]

test_dfs = [pd.read_csv(f, index_col='id') for f in test_prediction_files]

In [15]:
val_error = "XYZ"

In [16]:
filename="nasnetlarge_cv_f23_predictions_val_err={}.csv".format(val_error)

In [17]:
df_test_probas = pd.concat(test_dfs, axis=0)
y_test_probas = df_test_probas.groupby('id').mean()
y_test_pred = np.argmax(y_test_probas.values, axis=1) + 1   

sample_predicitions_df = pd.read_csv("/home/fast_storage/imaterialist-challenge-furniture-2018/sample_submission_randomlabel.csv", 
                                     index_col='id')
df = sample_predicitions_df.copy()
df.loc[y_test_probas.index, 'predicted'] = y_test_pred
df.to_csv(filename)

In [18]:
!{CD} python3 utils/update_test_predictions.py notebooks/{filename} notebooks/test_with_labels.csv notebooks/

In [19]:
!head -10 fixed_{filename}

id,predicted
1,12
2,71
3,91
4,54
5,126
6,76
7,94
8,8
9,127


id,predicted
1,12
2,71
3,91
4,54
5,126
6,76
7,94
8,8
9,127